In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
import random
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
train = train.drop('Unnamed: 0', axis=1)
test = test.drop('Unnamed: 0', axis=1)

In [5]:
y = train['0.1']
x = train.drop('0.1', axis=1)

In [6]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

x = sc.fit_transform(x)
test = sc.transform(test)

In [7]:
x_tensor = torch.FloatTensor(x)
y_tensor = torch.LongTensor(y)
test_tensor = torch.FloatTensor(test)

In [8]:
layer1 = torch.nn.Linear(x_tensor.shape[1], 256)
layer2 = torch.nn.Linear(256, 128)
layer3 = torch.nn.Linear(128, len(y_tensor.unique()))
ReLU = torch.nn.ReLU()
Softmax = torch.nn.Softmax(dim=1)

model = torch.nn.Sequential(layer1, ReLU, layer2, ReLU, layer3, Softmax)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

loss = torch.nn.CrossEntropyLoss()

In [9]:
epochs = 10000

for epoch in range(epochs):
    optimizer.zero_grad()
    hypothesis = model(x_tensor)
    cost = loss(hypothesis, y_tensor)
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'{epoch}/{epochs} cost: {cost.item()}')

0/10000 cost: 1.954034686088562
100/10000 cost: 1.7748022079467773
200/10000 cost: 1.7622977495193481
300/10000 cost: 1.7460623979568481
400/10000 cost: 1.7028226852416992
500/10000 cost: 1.6062690019607544
600/10000 cost: 1.571175217628479
700/10000 cost: 1.5452840328216553
800/10000 cost: 1.5020077228546143
900/10000 cost: 1.470703125
1000/10000 cost: 1.4509950876235962
1100/10000 cost: 1.4234778881072998
1200/10000 cost: 1.385155200958252
1300/10000 cost: 1.340802788734436
1400/10000 cost: 1.3126014471054077
1500/10000 cost: 1.300623893737793
1600/10000 cost: 1.294862985610962
1700/10000 cost: 1.2906709909439087
1800/10000 cost: 1.2837539911270142
1900/10000 cost: 1.2590715885162354
2000/10000 cost: 1.2422150373458862
2100/10000 cost: 1.2275103330612183
2200/10000 cost: 1.2064789533615112
2300/10000 cost: 1.189994215965271
2400/10000 cost: 1.1820014715194702
2500/10000 cost: 1.1782535314559937
2600/10000 cost: 1.17635178565979
2700/10000 cost: 1.1752445697784424
2800/10000 cost: 1.1

In [10]:
predict = torch.argmax(model(test_tensor), dim=1)
submit = pd.read_csv('submit_sample.csv')
submit['Category'] = predict.detach()
submit.to_csv('submit.csv', index=False)